In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
import seaborn as sns

# Author: Martin Luessi <mluessi@nmr.mgh.harvard.edu>
# License: BSD (3-clause)
from matplotlib.colors import TwoSlopeNorm

import os.path as op
from mne.time_frequency import tfr_multitaper
from mne.stats import permutation_cluster_1samp_test as pcluster_test

from mne_connectivity import spectral_connectivity_epochs, seed_target_indices
from mne.datasets import sample
from mne_connectivity.viz import plot_sensors_connectivity
from mne.stats import permutation_cluster_test
from scipy import stats as stats
from functools import partial

print(__doc__)

dict015= {'FP1':'eeg','FP2':'eeg','AF7':'eeg','AF3':'eeg','AFz':'eeg','AF4':'eeg','AF8':'eeg','F7':'eeg','F3':'eeg',
          'Fz':'eeg','F4':'eeg','F8':'eeg','FT9':'eeg','FC5':'eeg','FC1':'eeg','FC2':'eeg','FC6':'eeg','FT10':'eeg',
          'T7':'eeg','C3':'eeg','Cz':'eeg','C4':'eeg','T8':'eeg','TP9':'eeg','CP5':'eeg','CP1':'eeg', 'CP2':'eeg',
          'CP6':'eeg','TP10':'eeg','P7':'eeg','P3':'eeg','Pz':'eeg','P4':'eeg','P8':'eeg', 'O1':'eeg','O2':'eeg',
          'DBS1-2':'dbs','DBS2-3':'dbs','DBS3-4':'dbs','DBS4-5':'dbs','DBS5-6':'dbs','DBS6-7':'dbs','DBS7-8':'dbs',
          'mean(DBS1-DBS2, DBS2-DBS3, DBS3-DBS4, DBS4-DBS5, DBS5-DBS6, DBS6-DBS7, DBS7-DBS8)':'misc',
          'DynL(lc)':'misc','DynR(lc)':'misc','EmgL':'emg','EmgR':'emg','EmgL(lc)':'emg','EmgR(lc)':'emg'}

dict013_imp = {'FP1':'eeg','FP2':'eeg','AF7':'eeg','AF3':'eeg','AFz':'eeg','AF4':'eeg','AF8':'eeg','F7':'eeg','F3':'eeg',
          'Fz':'eeg','F4':'eeg','F8':'eeg','FC5':'eeg','FC1':'eeg','FC2':'eeg','FC6':'eeg',
          'T7':'eeg','C3':'eeg','Cz':'eeg','C4':'eeg','T8':'eeg','FT9':'eeg','CP5':'eeg','CP1':'eeg', 'CP2':'eeg',
          'CP6':'eeg','FT10':'eeg','TP9':'eeg','P3':'eeg','Pz':'eeg','P4':'eeg','P8':'eeg', 'O1':'eeg','O2':'eeg',
          'DBS1-2':'dbs','DBS2-3':'dbs','DBS3-4':'dbs','DBS4-5':'dbs','DBS5-6':'dbs','DBS6-7':'dbs','DBS7-8':'dbs',
          'mean(DBS1-DBS2, DBS2-DBS3, DBS3-DBS4, DBS4-DBS5, DBS5-DBS6, DBS6-DBS7, DBS7-DBS8)':'misc',
          'DynL(lc)':'misc','DynR(lc)':'misc','EmgL':'emg','EmgR':'emg','EmgL(lc)':'emg','EmgR(lc)':'emg'}

dict013_exp = dict015

dict011_imp = {'FP1':'eeg','FP2':'eeg','AF8':'eeg','AF4':'eeg','AFz':'eeg','AF3':'eeg','AF7':'eeg','F8':'eeg','F4':'eeg','Fz':'eeg','F3':'eeg',
              'F7':'eeg','FC6':'eeg','FC2':'eeg','FC1':'eeg','FC5':'eeg','T8':'eeg','C4':'eeg','Cz':'eeg','C3':'eeg','T7':'eeg','FT10':'eeg',
               'CP6':'eeg','CP2':'eeg','CP1':'eeg', 'CP5':'eeg', 'FT9':'eeg', 'TP10':'eeg','P4':'eeg','Pz':'eeg','P3':'eeg','P7':'eeg','O2':'eeg',
               'O1':'eeg', 'DBS1-2':'dbs','DBS2-3':'dbs','DBS3-4':'dbs','DBS4-5':'dbs','DBS5-6':'dbs','DBS6-7':'dbs','DBS7-8':'dbs',
              'mean(DBS1-DBS2, DBS2-DBS3, DBS3-DBS4, DBS4-DBS5, DBS5-DBS6, DBS6-DBS7, DBS7-DBS8)':'misc',
             'DynR(lc)':'misc', 'DynL(lc)':'misc','EmgR':'emg','EmgL':'emg','EmgR(lc)':'emg','EmgL(lc)':'emg'}

dict011_exp = {'FP1':'eeg','FP2':'eeg','AF8':'eeg','AF4':'eeg','AFz':'eeg','AF3':'eeg','AF7':'eeg','F8':'eeg','F4':'eeg','Fz':'eeg','F3':'eeg',
              'F7':'eeg','FT10':'eeg', 'FC6':'eeg','FC2':'eeg','FC1':'eeg','FC5':'eeg','FT9':'eeg','T8':'eeg','C4':'eeg','Cz':'eeg','C3':'eeg',
               'T7':'eeg','TP10':'eeg','CP6':'eeg','CP2':'eeg','CP1':'eeg', 'CP5':'eeg', 'TP9':'eeg', 'P8':'eeg','P4':'eeg','Pz':'eeg',
               'P3':'eeg','P7':'eeg','O2':'eeg', 'O1':'eeg', 'DBS1-2':'dbs','DBS2-3':'dbs','DBS3-4':'dbs','DBS4-5':'dbs','DBS5-6':'dbs',
               'DBS6-7':'dbs','DBS7-8':'dbs', 'mean(DBS1-DBS2, DBS2-DBS3, DBS3-DBS4, DBS4-DBS5, DBS5-DBS6, DBS6-DBS7, DBS7-DBS8)':'misc',
             'DynR(lc)':'misc', 'DynL(lc)':'misc','EmgR':'emg','EmgL':'emg','EmgR(lc)':'emg','EmgL(lc)':'emg'}
           
    
#'DBS1-234':'dbs','DBS234-567':'dbs','DBS567-8':'dbs','mean(DBS1-234, DBS234-567, DBS567-8)':'dbs',

dict08_imp = {'FP1':'eeg','FP2':'eeg','AF8':'eeg','AF4':'eeg','AFz':'eeg','AF3':'eeg','AF7':'eeg','F8':'eeg',
                'F4':'eeg','Fz':'eeg','F3':'eeg','F7':'eeg','FT10':'eeg','FC6':'eeg',
                'FC2':'eeg','FC1':'eeg','FC5':'eeg','FT9':'eeg','T8':'eeg','C4':'eeg','Cz':'eeg','C3':'eeg',
                'T7':'eeg','TP10':'eeg','CP6':'eeg','CP2':'eeg','CP1':'eeg','CP5':'eeg','TP9':'eeg','P4':'eeg',
                'Pz':'eeg','P3':'eeg','O2':'eeg','O1':'eeg',
                'DBS1-2':'dbs', 'DBS1-3':'dbs', 'DBS1-4':'dbs','DBS2-5':'dbs', 'DBS3-6':'dbs', 
                'DBS4-7':'dbs', 'DBS5-8':'dbs', 'DBS6-8':'dbs', 'DBS7-8':'dbs',              
                'mean(DBS1-2, DBS1-3, DBS1-4, DBS2-5, DBS3-6, DBS4-7, DBS5-8, DBS6-8, DBS7-8)':'misc',             
                'DynR(lc)':'misc',
                'DynL(lc)':'misc','EmgR':'emg','EmgL':'emg','EmgR(lc)':'emg','EmgL(lc)':'emg'}

dict08_exp = {'FP1':'eeg','FP2':'eeg','AF8':'eeg','AF4':'eeg','AFz':'eeg','AF3':'eeg','AF7':'eeg','F8':'eeg',
                'F4':'eeg','Fz':'eeg','F3':'eeg','F7':'eeg','FC6':'eeg', 'FC2':'eeg','FC1':'eeg','FC5':'eeg',
                'T8':'eeg','C4':'eeg','Cz':'eeg','C3':'eeg', 'T7':'eeg','FT10':'eeg', 'CP6':'eeg','CP2':'eeg',
                'CP1':'eeg','CP5':'eeg','FT9':'eeg','TP10':'eeg', 'P4':'eeg','Pz':'eeg','P3':'eeg','P7':'eeg',
                'O2':'eeg','O1':'eeg',
                'DBS1-2':'dbs', 'DBS1-3':'dbs', 'DBS1-4':'dbs','DBS2-5':'dbs', 'DBS3-6':'dbs', 
                'DBS4-7':'dbs', 'DBS5-8':'dbs', 'DBS6-8':'dbs', 'DBS7-8':'dbs',              
                'mean(DBS1-2, DBS1-3, DBS1-4, DBS2-5, DBS3-6, DBS4-7, DBS5-8, DBS6-8, DBS7-8)':'misc',         
                'DynR(lc)':'misc',
                'DynL(lc)':'misc','EmgR':'emg','EmgL':'emg','EmgR(lc)':'emg','EmgL(lc)':'emg'}

dict07_imp= {'FP1':'eeg','FP2':'eeg','AF8':'eeg','AF4':'eeg','AFz':'eeg','AF3':'eeg','AF7':'eeg','F8':'eeg',
                'F4':'eeg','Fz':'eeg','F3':'eeg','F7':'eeg','FT10':'eeg','FC6':'eeg',
                'FC2':'eeg','FC1':'eeg','FC5':'eeg','FT9':'eeg','T8':'eeg','C4':'eeg','Cz':'eeg','C3':'eeg',
                'T7':'eeg','TP10':'eeg','CP6':'eeg','CP2':'eeg','CP1':'eeg','CP5':'eeg','TP9':'eeg','P4':'eeg',
                'Pz':'eeg','P3':'eeg','O2':'eeg','O1':'eeg',
                'DBS1-2':'dbs', 'DBS1-3':'dbs', 'DBS1-4':'dbs','DBS2-5':'dbs', 'DBS3-6':'dbs',
                'DBS4-7':'dbs', 'DBS5-8':'dbs', 'DBS6-8':'dbs', 'DBS7-8':'dbs',
                'mean(DBS1-2, DBS1-3, DBS1-4, DBS2-5, DBS3-6, DBS4-7, DBS5-8, DBS6-8, DBS7-8)':'misc',
                'DynR':'misc','DynL':'misc','DynR(lc)':'misc',
                'DynL(lc)':'misc','EmgR':'emg','EmgL':'emg','EmgR(lc)':'emg','EmgL(lc)':'emg'}

dict07_exp= {'FP1':'eeg','FP2':'eeg','AF8':'eeg','AF4':'eeg','AFz':'eeg','AF3':'eeg','AF7':'eeg','F8':'eeg',
                'F4':'eeg','Fz':'eeg','F3':'eeg','F7':'eeg','FT10':'eeg','FC6':'eeg',
                'FC2':'eeg','FC1':'eeg','FC5':'eeg','FT9':'eeg','T8':'eeg','C4':'eeg','Cz':'eeg','C3':'eeg',
                'T7':'eeg','TP10':'eeg','CP6':'eeg','CP2':'eeg','CP1':'eeg','CP5':'eeg','TP9':'eeg','P8':'eeg','P4':'eeg',
                'Pz':'eeg','P3':'eeg','P7':'eeg','O2':'eeg','O1':'eeg',
                'DBS1-2':'dbs', 'DBS1-3':'dbs', 'DBS1-4':'dbs','DBS2-5':'dbs', 'DBS3-6':'dbs',
                'DBS4-7':'dbs', 'DBS5-8':'dbs', 'DBS6-8':'dbs', 'DBS7-8':'dbs',
                'mean(DBS1-2, DBS1-3, DBS1-4, DBS2-5, DBS3-6, DBS4-7, DBS5-8, DBS6-8, DBS7-8)':'misc',
                'DynR(lc)':'misc',
                'DynL(lc)':'misc','EmgR':'emg','EmgL':'emg','EmgR(lc)':'emg','EmgL(lc)':'emg'}

dict_eog= {'FP1':'eog'}


Automatically created module for IPython interactive environment


In [2]:
"""Average Baseline"""     
def average_baseline(epoch):

    epoch_norm1 =[] 
    for ii,ch in enumerate(epoch.ch_names):
        x = np.hstack(epoch.get_data()[:, ii, :]) # 2D dim = epoch* ch* time to 1D dim, freq * (time *epoch)

        x_mean=x.mean() # mean of all the freqs across all the complete experiment session.
        epoch_norm=[]    
        for jj in range(epoch.get_data()[:, ii, :].shape[0]):
            epoch_norm.append(((epoch.get_data()[jj, ii, :]-x_mean))) # applying normalaization on each trail
        epoch_norm1.append(epoch_norm)
    epoch_norm1 = np.array(epoch_norm1)
    epoch_norm1 = np.rollaxis(epoch_norm1,1) # normalized epoch data

    return mne.EpochsArray(epoch_norm1,  epoch.info, events=epoch.events, tmin=epoch.tmin)

In [8]:
base_dir = "/mnt/beegfs/malann/codes/Oddball_Project/Oddball Data/"
result_dir = "/mnt/beegfs/malann/codes/Oddball_Project/Results/"
subj_list = ['007']
#subj_list = ['015']

stage = ['implant']#, 'implant']explant
aff_cond = ['1', '101'];
aff_cond_plot = ['Standard', 'Oddball'];
hand = 'aff'

naff_cond = ['10', '110'];

for sub in subj_list:
    for st in stage:
        data_dir = base_dir+"EDEN"+sub+'/ANALYSIS/'
        fname = data_dir+"data_clean"+st+'.mat'
        
        if hand == 'naff':
            fname = data_dir+"data_clean"+st+'_naff_cond.mat'
            dyno = 'DynR(lc)'
        else:
            fname = data_dir+"data_clean"+st+'.mat'
            dyno = 'DynL(lc)'          

        
        if sub == '015' or sub=='014':
            dict1 = dict015
        elif sub == '013' and st== 'implant':
            dict1 = dict013_imp
        elif sub == '013' and st== 'explant':
            dict1 = dict013_exp
        elif sub == '011' and st== 'implant':
            dict1 = dict011_imp
        elif sub == '011' and st== 'explant':
            dict1 = dict011_exp
        elif sub == '008' and st== 'implant':
            dict1 = dict08_imp
        elif sub == '008' and st== 'explant':
            dict1 = dict08_exp
        elif sub == '007' and st== 'implant':
            dict1 = dict07_imp
        elif sub == '007' and st== 'explant':
            dict1 = dict07_exp

        info = mne.create_info(list(dict1.keys()), sfreq=200, ch_types='misc', verbose=None)
        epochs = mne.read_epochs_fieldtrip(fname,info,data_name='x', trialinfo_column=0)
        
        sfreq = epochs.info['sfreq']
        
        epochs.set_channel_types(dict1)

        kind='easycap-M1'
        mon = mne.channels.make_standard_montage(kind, head_size='auto')
        epochs.set_montage(mon,match_case=False, on_missing='ignore')
        
        epochs.filter(0.05,30)

        epochs_aff_odd = epochs[aff_cond[1]]
        epochs_aff_typ = epochs[aff_cond[0]]


        
        epochs_aff_typ_E = epochs_aff_typ.copy()
        epochs_aff_typ_NE = epochs_aff_typ.copy()
        
        # calculate outlier trial
        tmax = 6
        Dynmo_odd = epochs_aff_odd.copy().crop(-1,tmax).get_data(picks=dyno)
        Dynmo_odd = Dynmo_odd.reshape(Dynmo_odd.shape[0],Dynmo_odd.shape[2])
        
        t_target = int(200+200*1.5)
        #Calculate Error Rate
        ERR_value = []
        for i in range(len(Dynmo_odd)):
            ERR_value1 = ((Dynmo_odd[i,:].max()-Dynmo_odd[i,t_target])*100)/Dynmo_odd[i,t_target]
            ERR_value.append(ERR_value1)
        
        over_thres = 20
        
        outlier_idx=[]
        outlier_idx_odd_no_err=[]
        for j,m in enumerate(ERR_value):
            if (m>over_thres):
                outlier_idx.append(j)
            else:
                outlier_idx_odd_no_err.append(j)
        
        epochs_aff_odd_E=epochs_aff_odd[outlier_idx]
        epochs_aff_odd_NE=epochs_aff_odd[outlier_idx_odd_no_err]
        
        mne.epochs.equalize_epoch_counts([epochs_aff_odd_E,epochs_aff_typ_E], method='truncate')
        mne.epochs.equalize_epoch_counts([epochs_aff_odd_NE,epochs_aff_typ_NE], method='truncate')

        epochs_aff_odd_E.set_channel_types(dict_eog)
        epochs_aff_odd_NE.set_channel_types(dict_eog)
        epochs_aff_typ_E.set_channel_types(dict_eog)
        epochs_aff_typ_NE.set_channel_types(dict_eog)   


        base_min,base_max = -0.9,-0.4
        avg = False
        
        if avg == True:
            
            epochs_aff_odd_E = average_baseline(epochs_aff_odd_E)
            epochs_aff_odd_NE = average_baseline(epochs_aff_odd_NE)
            epochs_aff_typ_E = average_baseline(epochs_aff_typ_E)
            epochs_aff_typ_NE = average_baseline(epochs_aff_typ_NE)
        else:
            
            epochs_aff_odd_E = epochs_aff_odd_E.apply_baseline((base_min,base_max))
            epochs_aff_odd_NE = epochs_aff_odd_NE.apply_baseline((base_min,base_max))
            epochs_aff_typ_E = epochs_aff_typ_E.apply_baseline((base_min,base_max))
            epochs_aff_typ_NE = epochs_aff_typ_NE.apply_baseline((base_min,base_max))
    


        f_max_time = np.argmax(epochs_aff_odd_E.copy().crop(0,1).get_data(picks='DynL(lc)').reshape(epochs_aff_odd_E.copy().crop(0,1).get_data(picks='DynL(lc)').shape[0],epochs_aff_odd_E.copy().crop(0,1).get_data(picks='DynL(lc)').shape[2]).mean(axis=0))/200

        
        # Pick Ipsilesional and DBS channels only
        seed_chs = epochs_aff_odd_E.copy().pick('dbs').ch_names
        
        ch_picks =['Fz','F4','F8','FC2','FC6','Cz', 'C4','CP2','CP6','Pz','P4']
        #ch_picks = ['FC2', 'FC1', 'Cz', 'Fz', 'CP2', 'CP1', 'Pz']


        
        ch_picks.extend(seed_chs)
        epochs_aff_odd_E.pick(ch_picks)
        epochs_aff_typ_E.pick(ch_picks)
        epochs_aff_odd_NE.pick(ch_picks)
        epochs_aff_typ_NE.pick(ch_picks)
        
        rng = np.random.default_rng(42)  # set a random seed
        
        tmin_coh_lst = [0] #0 , f_max_time # Error detection, Error correction, Full time
        tmax_coh_lst = [1.5] #1.5 f_max_time, 
            
        for tmin_coh, tmax_coh in zip(tmin_coh_lst, tmax_coh_lst):
            coh_diff_E_permuted = []
            coh_diff_NE_permuted = []
            
            n_perms = 2000
            
            for iperm in range(n_perms):
            # Define different time windows to look for coherence in Error detection and Error correction
            

                picks = mne.pick_types(epochs_aff_odd_E.info, eeg=True, dbs=True, emg= False,stim=False, eog=False,
                                       exclude='bads')
            
                epochs_aff_odd_E.pick(['eeg','dbs'])
                epochs_aff_typ_E.pick(['eeg','dbs'])
                epochs_aff_odd_NE.pick(['eeg','dbs'])
                epochs_aff_typ_NE.pick(['eeg','dbs'])
                
                # Use 'DBS' channels as seed
                        
                seed_chs = epochs_aff_odd_E.copy().pick('dbs').ch_names
                coh_E_diff = []
                coh_NE_diff = []
                for n_seed, seed_ch in enumerate(seed_chs):
                    
                    picks_ch_names = epochs_aff_odd_E.ch_names
                    # Create seed-target indices for connectivity computation
                    seed = len(epochs_aff_odd_E.ch_names)-1
                    targets = np.arange(len(picks))
                    indices = seed_target_indices(seed, targets) 



                    # permuting for oddball only seed dbs channel   
                    ep_dbs_seed = epochs_aff_odd_E.copy().pick([seed_ch])
                    perm = rng.permutation(len(ep_dbs_seed))
                    epochs_in_permuted_order = ep_dbs_seed[perm]
                    random_dbs = epochs_in_permuted_order.get_data()
                    epochs_aff_odd_E_drop_seed = epochs_aff_odd_E.copy().drop_channels([seed_ch]).get_data()                
                    new_epochs_aff_odd_E = np.concatenate((epochs_aff_odd_E_drop_seed, random_dbs), axis=1)
                    new_epochs_aff_odd_E = mne.EpochsArray(new_epochs_aff_odd_E, epochs_aff_odd_E.info, events=epochs_aff_odd_E.events, 
                                                           tmin=epochs_aff_odd_E.tmin)

            
                    # permuting for standard only seed dbs channel            
                    ep_typ_E_dbs_seed = epochs_aff_typ_E.copy().pick([seed_ch])            
                    epochs_typ_E_in_permuted_order = ep_typ_E_dbs_seed[perm]
                    random_typ_E_dbs = epochs_typ_E_in_permuted_order.get_data()                    
                    epochs_aff_typ_E_drop_seed = epochs_aff_typ_E.copy().drop_channels([seed_ch]).get_data()                
                    new_epochs_aff_typ_E = np.concatenate((epochs_aff_typ_E_drop_seed, random_typ_E_dbs), axis=1)
                    new_epochs_aff_typ_E = mne.EpochsArray(new_epochs_aff_typ_E, epochs_aff_typ_E.info, events=epochs_aff_typ_E.events, 
                                                          tmin=epochs_aff_typ_E.tmin)
            
                    
                    
                    # permuting for oddball only seed dbs channel   
                    ep_dbs_seed_NE = epochs_aff_odd_NE.copy().pick([seed_ch])
                    perm = rng.permutation(len(ep_dbs_seed_NE))
                    epochs_in_permuted_order_NE = ep_dbs_seed_NE[perm]
                    random_dbs_NE = epochs_in_permuted_order_NE.get_data()
                    epochs_aff_odd_NE_drop_seed = epochs_aff_odd_NE.copy().drop_channels([seed_ch]).get_data()                
                    new_epochs_aff_odd_NE = np.concatenate((epochs_aff_odd_NE_drop_seed, random_dbs_NE), axis=1)
                    new_epochs_aff_odd_NE = mne.EpochsArray(new_epochs_aff_odd_NE, epochs_aff_odd_NE.info, events=epochs_aff_odd_NE.events, 
                                                           tmin=epochs_aff_odd_NE.tmin)


                
                    # permuting for standard only seed dbs channel            
                    ep_typ_NE_dbs_seed = epochs_aff_typ_NE.copy().pick([seed_ch])            
                    epochs_typ_NE_in_permuted_order = ep_typ_NE_dbs_seed[perm]
                    random_typ_NE_dbs = epochs_typ_NE_in_permuted_order.get_data()                    
                    epochs_aff_typ_NE_drop_seed = epochs_aff_typ_NE.copy().drop_channels([seed_ch]).get_data()                
                    new_epochs_aff_typ_NE = np.concatenate((epochs_aff_typ_NE_drop_seed, random_typ_NE_dbs), axis=1)
                    new_epochs_aff_typ_NE = mne.EpochsArray(new_epochs_aff_typ_NE, epochs_aff_typ_NE.info, events=epochs_aff_typ_NE.events, 
                                                          tmin=epochs_aff_typ_NE.tmin)

                    
                
                    freq_band = ['Theta', 'Alpha', 'Low beta', 'High beta']
                    min_freq = (4, 8,13,21)
                    max_freq = (7, 12,20,30)    
                    
                    # Define wavelet frequencies and number of cycles
                    cwt_freqs = np.arange(1, 41, 1)
                    cwt_n_cycles = 7
                
                    # Run the connectivity analysis using 2 parallel jobs
                    con_odd_E = spectral_connectivity_epochs(
                        new_epochs_aff_odd_E, indices=indices,
                        method='coh', mode='cwt_morlet', sfreq=sfreq, fmin = min_freq, fmax = max_freq, faverage =True, tmin = tmin_coh, tmax =tmax_coh,
                        cwt_freqs=cwt_freqs, cwt_n_cycles=cwt_n_cycles, n_jobs=30)
                
                    con_typ_E = spectral_connectivity_epochs(
                        new_epochs_aff_typ_E, indices=indices,
                        method='coh', mode='cwt_morlet', sfreq=sfreq, fmin = min_freq, fmax = max_freq, faverage =True, tmin = tmin_coh, tmax =tmax_coh,
                        cwt_freqs=cwt_freqs, cwt_n_cycles=cwt_n_cycles, n_jobs=30)
            
                    con_odd_NE = spectral_connectivity_epochs(
                        new_epochs_aff_odd_NE, indices=indices,
                        method='coh', mode='cwt_morlet', sfreq=sfreq, fmin = min_freq, fmax = max_freq, faverage =True, tmin = tmin_coh, tmax =tmax_coh,
                        cwt_freqs=cwt_freqs, cwt_n_cycles=cwt_n_cycles, n_jobs=30)
                
                    con_typ_NE = spectral_connectivity_epochs(
                        new_epochs_aff_typ_NE, indices=indices,
                        method='coh', mode='cwt_morlet', sfreq=sfreq, fmin = min_freq, fmax = max_freq, faverage =True, tmin = tmin_coh, tmax =tmax_coh,
                        cwt_freqs=cwt_freqs, cwt_n_cycles=cwt_n_cycles, n_jobs=30)
            
                    
                    times = con_odd_E.times
                    freqs = con_odd_E.freqs
                
                    # Mark the seed channel with a value of 1.0, so we can see it in the plot
                    # con_odd.get_data()[np.where(indices[1] == seed)] = 1.0
                    # con_typ.get_data()[np.where(indices[1] == seed)] = 1.0
                
                    #layout = mne.find_layout(epochs.info, 'eeg')  # use full layout
                    tfr_odd_E = mne.time_frequency.AverageTFR(epochs_aff_odd_E.info, con_odd_E.get_data(), times, freqs, len(epochs_aff_odd_E))
                    tfr_odd_E.save(data_dir+'tfr_odd_E'+st+seed_ch+'-tfr.h5',overwrite =True)
                
                    tfr_typ_E = mne.time_frequency.AverageTFR(epochs_aff_typ_E.info, con_typ_E.get_data(), times, freqs, len(epochs_aff_typ_E))
                    tfr_typ_E.save(data_dir+'tfr_typ_E'+st+seed_ch+'-tfr.h5',overwrite =True)
            
                    #layout = mne.find_layout(epochs.info, 'eeg')  # use full layout
                    tfr_odd_NE = mne.time_frequency.AverageTFR(epochs_aff_odd_NE.info, con_odd_NE.get_data(), times, freqs, len(epochs_aff_odd_NE))
                    tfr_odd_NE.save(data_dir+'tfr_odd_NE'+st+seed_ch+'-tfr.h5',overwrite =True)
                
                    tfr_typ_NE = mne.time_frequency.AverageTFR(epochs_aff_typ_NE.info, con_typ_NE.get_data(), times, freqs, len(epochs_aff_typ_NE))
                    tfr_typ_NE.save(data_dir+'tfr_typ_NE'+st+seed_ch+'-tfr.h5',overwrite =True)
                    
                    
                    # TFR differences in freq bands and time period 0 to 1.5s
                
                    tfr_E = tfr_odd_E- tfr_typ_E
                    tfr_NE = tfr_odd_NE- tfr_typ_NE
                
                    freq_band = ['Theta', 'Alpha', 'Low beta', 'High beta']
                        
                    n_channels = len(tfr_E.copy().pick('eeg').ch_names)
                    X1 = []
                    X2 = []
                    for n, band in enumerate(freq_band):
                        tfr_E_ch = []
                        tfr_NE_ch = []
                        for ch in range(n_channels):
                            tfr_E_ch.append((tfr_E.data[:,:,:].mean(axis=2)[ch,n]))
                            tfr_NE_ch.append((tfr_NE.data[:,:,:].mean(axis=2)[ch,n]))
            
                        tfr_E_ch = np.array(tfr_E_ch).reshape(n_channels,1)   
                        tfr_NE_ch = np.array(tfr_NE_ch).reshape(n_channels,1)   
            
                        X1.append(tfr_E_ch)
                        X2.append(tfr_NE_ch)
                
                    coh_E_diff.append(X1)
                    coh_NE_diff.append(X2)
                coh_diff_E_permuted.append(coh_E_diff)
                coh_diff_NE_permuted.append(coh_NE_diff)
                
            coh_diff_E_permuted1 = np.array(coh_diff_E_permuted)
            coh_diff_NE_permuted1 = np.array(coh_diff_NE_permuted)
    
            file = f"{result_dir}coh_diff_E_permuted_{n_perms}_{sub}_{st}_{tmin_coh}_{tmax_coh}_center"
    
            np.save(file, coh_diff_E_permuted1)
            
            file = f"{result_dir}coh_diff_NE_permuted_{n_perms}_{sub}_{st}_{tmin_coh}_{tmax_coh}_center"
    
            np.save(file, coh_diff_NE_permuted1)

Adding metadata with 9 columns
600 matching events found
No baseline correction applied
Setting up band-pass filter from 0.05 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.05 Hz (-6 dB cutoff frequency: 0.03 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 13201 samples (66.005 s)



/tmp/ipykernel_3715812/122343463.py:49: RuntimeWarning: The unit for channel(s) AF3, AF4, AF7, AF8, AFz, C3, C4, CP1, CP2, CP5, CP6, Cz, DBS1-2, DBS2-3, DBS3-4, DBS4-5, DBS5-6, DBS6-7, DBS7-8, EmgL, EmgL(lc), EmgR, EmgR(lc), F3, F4, F7, F8, FC1, FC2, FC5, FC6, FP1, FP2, FT10, FT9, Fz, O1, O2, P3, P4, P7, Pz, T7, T8, TP10 has changed from NA to V.
  epochs.set_channel_types(dict1)
/tmp/ipykernel_3715812/122343463.py:55: RuntimeWarning: filter_length (13201) is longer than the signal (1801), distortion is likely. Reduce filter length or filter a longer signal.
  epochs.filter(0.05,30)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 881 tasks      | 

KeyboardInterrupt: 